In [47]:
from landlordai.game.player import LearningPlayer, LearningPlayer_v2
from landlordai.sim.simulate import Simulator, NoRecordsException
from landlordai.sim.game_stats import GameStats

import numpy as np
from tqdm.notebook import tqdm_notebook
import pickle
import subprocess

import pkg_resources
pkg_resources.get_distribution('landlord-ai').version

'0.1.32'

In [48]:
!rm -r models
!gsutil -m cp -r gs://landlord_ai/models .

/snap/google-cloud-sdk/126/lib/third_party/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
Copying gs://landlord_ai/models/4_10_actualq1_model0/history.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model0/position.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model1/position.h5...             
Copying gs://landlord_ai/models/4_10_actualq1_model1/history.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model10/history.h5...             
Copying gs://landlord_ai/models/4_10_actualq1_model10/position.h5...            
Copying gs://landlord_ai/models/4_10_actualq1_model14/history.h5...
Copying gs://landlord_ai/models/4_10_actualq1_model14/position.h5...            
Copying gs://landlord_ai/models/4_10_actualq1_model15/history.h5...             
Copying gs://landlord_ai/models/4_10_actualq1_model15/position.h5...
Copying gs://landlord_ai/models/4_10_actualq1_mod

Copying gs://landlord_ai/models/4_11_actualq1_model5/position.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model0/history.h5...
Copying gs://landlord_ai/models/4_11_actualq2_model1/history.h5...              
Copying gs://landlord_ai/models/4_11_actualq2_model0/position.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model1/position.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model10/history.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model12/history.h5...             
Copying gs://landlord_ai/models/4_11_actualq2_model11/history.h5...
Copying gs://landlord_ai/models/4_11_actualq2_model11/position.h5...
Copying gs://landlord_ai/models/4_11_actualq2_model12/position.h5...            
Copying gs://landlord_ai/models/4_11_actualq2_model10/position.h5...            
Copying gs://landlord_ai/models/4_11_actualq2_model2/history.h5...              
Copying gs://landlord_ai/models/4_11_actualq2_model20/history.h5...

Copying gs://landlord_ai/models/4_9_actualq3_model22/history.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model24/position.h5...
Copying gs://landlord_ai/models/4_9_actualq3_model22/position.h5...
Copying gs://landlord_ai/models/4_9_actualq3_model24/history.h5...
Copying gs://landlord_ai/models/4_9_actualq3_model23/history.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model23/position.h5...             
Copying gs://landlord_ai/models/4_9_actualq3_model3/history.h5...               
Copying gs://landlord_ai/models/4_9_actualq3_model3/position.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model31/history.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model30/history.h5...              
Copying gs://landlord_ai/models/4_9_actualq3_model31/position.h5...             
Copying gs://landlord_ai/models/4_9_actualq3_model30/position.h5...             
Copying gs://landlord_ai/models/4_9_actualq3_model32/history.h5...  

In [49]:
models_dir = 'models/'
def refresh_model_pool():
    try:
        subprocess.check_output(['rm', '-r', 'models'])
    except Exception as e:
        print(e)
        
    subprocess.check_output(['gsutil', '-m', 'cp', '-r', 'gs://landlord_ai/models', '.'])

    
    player_models = [file for file in os.listdir(models_dir) if '4_11_actualq3' in file]
    #competitor_models = ['4_9_actualq2_model30', '4_9_actualq1_model21', '4_9_actualq3_model1', '4_9_actualq4_model20'] + [
    #    '4_5_mc1_model10', '4_2_sim4_model15', '4_10_actualq1_model10'
    #]
    competitor_models = []
    
    player_pool = create_pool_from_models(player_models)
    competitor_pool = create_pool_from_models(competitor_models, estimation_mode=LearningPlayer.NO_ESTIMATION)
    
    return player_pool, competitor_pool

    
def create_pool_from_models(models, estimation_mode=LearningPlayer.ACTUAL_Q):
    pool = []
    for model in models:
        pool.append(LearningPlayer(name=model, 
                                          epsilon=0.05,
                                          net_dir=models_dir + model,
                                          estimation_mode=LearningPlayer.ACTUAL_Q,
                                          discount_factor=1,
                                          learning_rate=0.2))

    return pool
    


In [50]:
player_pool, competitor_pool = refresh_model_pool()
player_pool_bc = sc.broadcast(player_pool)
competitor_pool_bc = sc.broadcast(competitor_pool)

/opt/conda/anaconda/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [51]:
for player in player_pool:
    print(player.get_name())

4_11_actualq3_model0
4_11_actualq3_model10
4_11_actualq3_model20


In [52]:
data_folder = '4_11_actualq4'

In [53]:
#!rm -r $data_folder
!mkdir $data_folder
#!gsutil rm -r gs://landlord_ai/$data_folder

In [54]:
cores_per_executor = 2

# since each spark executor fights for cpu space if we run a bunch of threads
def limit_threads(num_threads=2):
    import tensorflow as tf
    from tensorflow.python.eager import context

    _ = tf.Variable([1])

    context._context = None
    context._create_context()

    tf.config.threading.set_inter_op_parallelism_threads(num_threads)
    tf.config.threading.set_intra_op_parallelism_threads(num_threads)

In [55]:
num_games_per = 5000
parallel_core = sc.parallelize(zip(range(num_games_per), range(num_games_per))).repartition(2).cache()
_ = parallel_core.collect()

In [56]:
def run_sim(x):
    # ensure proper random seeding
    import numpy as np
    import random
    random.seed(x)
    np.random.seed(x)
    
    # set the number of threads once per machine
    import tensorflow as tf
    current_threads = tf.config.threading.get_inter_op_parallelism_threads()
    if current_threads != cores_per_executor:
        limit_threads(cores_per_executor)
    
    # simulate games
    from landlordai.sim.simulate import Simulator

    while True:
        sim = Simulator(2, player_pool_bc.value, competitor_pool_bc.value, record_loser_pct=1.0)
        sim.play_rounds(debug=False)
        try:
            return sim.get_sparse_game_data(), sim.get_results()
        except NoRecordsException:
            pass

def reduce_data(x, y):
    sparse_data_left, result_pairs_left = x
    sparse_data_right, result_pairs_right = y
    # history matrix, move vectors, hand vectors, q's
    sparse_data_merged = (sparse_data_left[0] + sparse_data_right[0], 
                          np.vstack([sparse_data_left[1], sparse_data_right[1]]), 
                          np.vstack([sparse_data_left[2], sparse_data_right[2]]), 
                          np.hstack([sparse_data_left[3], sparse_data_right[3]]))
    game_results_merged = result_pairs_left + result_pairs_right
    return sparse_data_merged, game_results_merged

In [ ]:

for i in tqdm_notebook(range(100)):
    # refresh models as they're being trained
    if i % 5 == 4:
        player_pool, competitor_pool = refresh_model_pool()
        player_pool_bc = sc.broadcast(player_pool)
        competitor_pool_bc = sc.broadcast(competitor_pool)
    
    sparse_data, _ = parallel_core \
                    .partitionBy(num_games_per, lambda x: x) \
                    .map(lambda x : run_sim(x[0] + (i * num_games_per))).reduce(reduce_data)
    
    filename = data_folder + '/' + str(i) + '.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(sparse_data, f)
    
    subprocess.check_output(['gsutil', 'cp', filename, 'gs://landlord_ai/' + filename])

/opt/conda/anaconda/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/opt/conda/anaconda/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/opt/conda/anaconda/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/opt/conda/anaconda/lib/python3.6/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file